In [3]:
# get ic decay
from get_data import get_data,bars
from singletrader.processors.cs_processor import CsWinzorize
from singletrader.shared.utility import save_pkl
from singletrader.factorlib import FactorEvaluation,summary_plot
import pandas as pd
import plotly.express as px
from plotly.figure_factory import create_table
import warnings
warnings.filterwarnings('ignore')
from singletrader.processors.cs_processor import IndAggregation,CsWinzorize,CsStandardize
from singletrader.constant import get_industry_cons
import pandas as pd

In [2]:
data = get_data().dropna()
Ind_info = get_industry_cons(name='sw_l2')
# 获取因子列表，剔除行情数据
features = [_f for _f in data.columns if _f not in bars] + ['ep']#['circulating_market_cap']
features_with_mv = features + ['circulating_market_cap']


cs_win = CsWinzorize(method='qtile',k=0.01) # 极值处理器
cs_std = CsStandardize() # 标准化处理器 默认z-socre
ind_agg = IndAggregation() #创建IndAggregation对象（自建类用于行业聚合处理）

# mom1M收益合成行业价格序列（cap）
return1m_ind_agg_cap = ind_agg(data[['mom1M','circulating_market_cap']],weights='circulating_market_cap',ind_info=Ind_info)
price_ind_cap = (1+return1m_ind_agg_cap['mom1M'].unstack()).cumprod().stack()



# mom1M收益合成行业价格序列（eq）
return1m_ind_agg_eq = ind_agg(data[['mom1M']],weights='eq',ind_info=Ind_info)
price_ind_eq = (1+return1m_ind_agg_eq['mom1M'].unstack()).cumprod().stack()


data[features] = cs_win(data[features]) # 极值处理
# data[features] = cs_std(data[features]) # z-score



# 因子数据聚合(eq)
ind_agg_data_eq = ind_agg(data,weights='eq',ind_info=Ind_info)[features]
# 因子数据聚合(cap)
ind_agg_data_cap = ind_agg(data,weights='circulating_market_cap',ind_info=Ind_info)[features]

# 将合成的价格数据转为pd.DataFrame
price_ind_eq = pd.DataFrame(price_ind_eq,columns=['close'])
price_ind_cap = pd.DataFrame(price_ind_cap,columns=['close'])




# Tips
-   因子处理在聚合前完成，聚合后的行业级别因子未进行任何处理
-   行业的价格数据时使用行业成分股分别根据等权和市值加权收益合成
-   在计算市值加权的聚合因子时，使用当月收盘市值作为权重计算依据
-   历史行业信息缺失，用当前最新状态下的行业信息往前填充
-   共计31个行业，groups = 10意味最高/最低组别拥有3-4个行业

In [3]:
# 行业分布情况
px.bar(Ind_info.groupby(Ind_info).count().sort_values(ascending=False),title='count of different industries')

# industry aggregation summary(eq)

In [4]:
fe_eq = FactorEvaluation(bar_data=price_ind_eq,factor_data=ind_agg_data_eq,industry_neutralize=False,winzorize=False,standardize=False,freq=12)
fe_eq.get_summary(holding_period=1,groups=10,add_shift=0,start_date='2010-01-01',end_date='2022-12-31',base='close')

start_date:2010-01-01 / end_date:2022-12-31


,skew,adjskew,distance,mom6x3,mom9x3,mom12x3,mom1M,mom2M,mom3M,mom6M,turnover3M,amount3M,stddev_diff,stddev,ep
ic.mean,-0.0151,-0.0139,-0.0236,0.0105,0.0308,0.0384,-0.0087,-0.0239,-0.0207,-0.0118,-0.0266,-0.0266,0.0093,-0.0142,-0.0094
ic.t-stats,-1.1654,-1.0935,-1.2582,0.5435,1.5870,1.9867,-0.4563,-1.1596,-1.1128,-0.5962,-1.1787,-1.4663,0.6966,-0.6530,-0.5408
AnnRet_Long,-0.0354,-0.0281,-0.0239,0.0108,0.0222,0.0352,-0.0117,-0.0165,0.0042,-0.0116,-0.0072,-0.0234,0.0165,-0.0012,-0.0419
AnnRet_short,0.0140,0.0297,0.0276,0.0062,-0.0548,-0.0475,0.0095,0.0098,-0.0000,-0.0112,-0.0015,0.0370,-0.0331,-0.0170,-0.0034
AnnRet,-0.0494,-0.0578,-0.0515,0.0046,0.0770,0.0827,-0.0212,-0.0264,0.0042,-0.0005,-0.0057,-0.0604,0.0496,0.0158,-0.0385
SR_Long,-0.3966,-0.3548,-0.2766,0.0888,0.2011,0.3106,-0.0903,-0.1271,0.0358,-0.0932,-0.0646,-0.1876,0.1997,-0.0097,-0.3103
SR_Short,0.1876,0.3992,0.2301,0.0696,-0.6166,-0.5682,0.1069,0.1088,-0.0002,-0.1269,-0.0116,0.6017,-0.4091,-0.1595,-0.0519
SR,-0.3625,-0.4659,-0.2768,0.0246,0.4421,0.4642,-0.1107,-0.1356,0.0251,-0.0024,-0.0261,-0.3692,0.3690,0.0774,-0.2733
TO_Long,0.8194,0.8223,0.2948,0.4888,0.3697,0.3261,0.8347,0.6074,0.4963,0.3692,0.2437,0.1335,0.9320,0.5950,0.0685
TO_Short,0.8284,0.8300,0.3643,0.5524,0.4095,0.3457,0.8715,0.6733,0.5478,0.4048,0.1489,0.2280,0.9345,0.4433,0.1125


# industry aggregation summary(cap)

In [5]:
fe_cap = FactorEvaluation(bar_data=price_ind_cap,factor_data=ind_agg_data_cap,industry_neutralize=False,winzorize=False,standardize=False,freq=12)
fe_cap.get_summary(holding_period=1,groups=10,add_shift=0,start_date='2010-01-01',end_date='2022-12-31',base='close')

start_date:2010-01-01 / end_date:2022-12-31


,skew,adjskew,distance,mom6x3,mom9x3,mom12x3,mom1M,mom2M,mom3M,mom6M,turnover3M,amount3M,stddev_diff,stddev,ep
ic.mean,-0.0017,0.0002,-0.0219,0.0164,0.0352,0.0499,-0.0008,-0.0204,-0.0099,0.0016,-0.0050,-0.0372,0.0058,0.0131,-0.0178
ic.t-stats,-0.1362,0.0163,-1.1391,0.8878,1.8743,2.6348,-0.0428,-1.0275,-0.5334,0.0832,-0.2236,-2.0995,0.4528,0.5993,-1.0202
AnnRet_Long,-0.0199,-0.0054,-0.0577,0.0250,0.0116,0.0558,-0.0085,-0.0303,0.0030,-0.0306,0.0239,-0.0674,0.0042,0.0078,-0.0676
AnnRet_short,-0.0124,-0.0073,-0.0024,-0.0490,-0.0958,-0.0793,-0.0290,-0.0068,-0.0074,-0.0210,-0.0630,0.0276,-0.0308,-0.0675,-0.0293
AnnRet,-0.0075,0.0019,-0.0553,0.0740,0.1073,0.1352,0.0205,-0.0235,0.0104,-0.0096,0.0870,-0.0950,0.0350,0.0753,-0.0384
SR_Long,-0.1948,-0.0501,-0.5431,0.1752,0.0842,0.4167,-0.0603,-0.1939,0.0213,-0.2067,0.1729,-0.4636,0.0431,0.0502,-0.3976
SR_Short,-0.1480,-0.0981,-0.0170,-0.5510,-0.9542,-0.8141,-0.2956,-0.0599,-0.0779,-0.2171,-0.3875,0.3202,-0.3382,-0.5166,-0.3508
SR,-0.0482,0.0129,-0.2468,0.3611,0.5201,0.6562,0.0983,-0.1012,0.0507,-0.0430,0.3133,-0.4669,0.2407,0.3010,-0.2039
TO_Long,0.8432,0.8392,0.2903,0.5032,0.3831,0.3345,0.8486,0.6263,0.5161,0.3901,0.2079,0.1320,0.9355,0.6015,0.0625
TO_Short,0.8392,0.8276,0.3442,0.5525,0.3982,0.3272,0.8664,0.6590,0.5562,0.4060,0.0885,0.2023,0.9304,0.3816,0.0917


In [9]:
l1 = get_industry_cons(name = 'sw_l1')
l2 = get_industry_cons(name = 'sw_l2')
l3 = get_industry_cons(name = 'sw_l3')

In [19]:
import jqdatasdk as jq
import os        
jq.auth(os.environ.get('JQ_USER') ,
            os.environ.get('JQ_PASSWD'))
ind_l1 = jq.get_industries(name='sw_l1', date=None).set_index('name')
ind_l2 = jq.get_industries(name='sw_l2', date=None).set_index('name')
ind_l3 = jq.get_industries(name='sw_l3', date=None).set_index('name')
ind_l1['company numbers'] = l1.groupby(l1).count()
ind_l2['company numbers'] = l2.groupby(l2).count()
ind_l3['company numbers'] = l3.groupby(l3).count()



AttributeError: 'datetime.date' object has no attribute 'date'

In [18]:
writer=pd.ExcelWriter('申万行业分类.xlsx')
ind_l1.to_excel(excel_writer=writer,sheet_name='level_1')
ind_l2.to_excel(excel_writer=writer,sheet_name='level_2')
ind_l3.to_excel(excel_writer=writer,sheet_name='level_3')
writer.save()

In [14]:
l1.groupby(l1).count()

industry_name
交通运输I    125
休闲服务I     73
传媒I      140
公用事业I    123
农林牧渔I    102
化工I      375
医药生物I    465
商业贸易I    104
国防军工I    125
家用电器I     83
建筑材料I     75
建筑装饰I    160
房地产I     115
有色金属I    135
机械设备I    483
汽车I      252
煤炭I       38
环保I      127
电子I      420
电气设备I    317
石油石化I     47
纺织服装I    114
综合I       24
美容护理I     29
计算机I     329
轻工制造I    149
通信I      121
钢铁I       46
银行I       42
非银金融I     89
食品饮料I    119
Name: industry_name, dtype: int64

In [15]:
ind_l1

,name,start_date
801740,国防军工I,2014-02-21
801020,采掘I,2004-02-10
801110,家用电器I,2004-02-10
801160,公用事业I,2004-02-10
801060,建筑建材I,2004-02-10
801770,通信I,2014-02-21
801010,农林牧渔I,2004-02-09
801120,食品饮料I,2004-02-10
801750,计算机I,2014-02-21
801050,有色金属I,2004-02-10
